In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Extraction de thèmes sur les questions d'immigration

Il y a 4 questions sur le sujet de l'immigration : 

'Q141 - Que pensez-vous de la situation de l'immigration en France aujourd'hui et de la politique migratoire ? Quelles sont, selon vous, les critères à mettre en place pour définir la politique migratoire ?'

'Q142 - En matière d'immigration, une fois nos obligations d'asile remplies, souhaitez-vous que nous puissions nous fixer des objectifs annuels définis par le Parlement ?'

'Q143 - Que proposez-vous afin de répondre à ce défi qui va durer ?'

'Q144 - Quelles sont, selon vous, les modalités d'intégration les plus efficaces et les plus justes à mettre en place aujourd'hui dans la société ?'

In [2]:
df = pd.read_csv('../data/DEMOCRATIE_ET_CITOYENNETE.csv', index_col=0)

In [20]:
texts = (df.iloc[:, -2]+' '+df.iloc[:, -3]+' '+df.iloc[:, -4]+' '+df.iloc[:, -5]).dropna().apply(lambda x: x.lower()).values


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

no_features = 1000
stop_words = ['le', 'la', 'ces', 'oui', 'est', 'de', 'qui', 'et', 'se', 'il', 'pour', 'un', 'des', 'une', 'sur', 'ou', 'je', 'les', 'que', 'dans', 'suis', 'sont', 'au', 'en', 'on', 'par', 'ai', 'du', 'ce', 'mais', 'ne', 'pas', 'plus', 'qu', 'être', 'leur', 'aux', 'mon', 'moi', 'me', 'ma', 'son', 'avoir', 'car', 'nous', 'ils', 'etc', 'vous', 'faire', 'si']

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=stop_words)
tfidf = tfidf_vectorizer.fit_transform(texts)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=stop_words)
tf = tf_vectorizer.fit_transform(texts)
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

no_topics = 10

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [12]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 20
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

Topic 0:
est france elle problème bien cela comme doit fait question cette tout même politique très avec sans peut donc situation
Topic 1:
français droit asile france trop nationalité étrangers aides sol française immigrés regroupement familial parler obligation ans frontières supprimer doit sociales
Topic 2:
pays aider origine développement leurs aide avec développer guerre retour économique pauvres concernés populations europe afrique départ afin autres émigration
Topic 3:
politique accueil langue intégration apprentissage formation française migratoire avec éducation oui emploi européenne travail non doit logement critères niveau réfugiés
Topic 4:
immigration choisie économique faut stopper limiter trop non oui clandestine illégale subie canada frontières immigrés zéro contre contrôlée légale politique
Topic 5:
notre nos valeurs lois langue respect culture vie respecter société république coutumes histoire mode pays apprendre besoins règles accepter obligations
Topic 6:
faut accueil

In [110]:
first_topic = pd.DataFrame(nmf.transform(tfidf)).apply(np.argmax, axis=1)

/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:51: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)


In [111]:
first_topic.groupby(first_topic.values).count()

0    6727
1    2757
2    4177
3    2462
4    1981
5    1652
6    1475
7    1861
8     738
9    2920
dtype: int64

In [112]:
pd.DataFrame(nmf.transform(tfidf)).apply(np.argmax)

/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:51: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)


0     2575
1    16273
2    20422
3    20273
4    19584
5    21747
6    18007
7     2370
8      285
9    20377
dtype: int64